# Final Notebook

---

In [ ]:
# Import necessary libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier # Binary dependent variable
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import AdaBoostClassifier
from datetime import datetime
from xgboost import XGBClassifier
import joblib # to save trained model 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
import shap 
import dill # to save workspace
from sklearn.inspection import permutation_importance # permutation feature importance for global interpretability
from sklearn.inspection import PartialDependenceDisplay as pdp # partial dependence plots for global interpretability
from sklearn.metrics import accuracy_score, r2_score
from sklearn.tree import plot_tree
from lime import lime_tabular # Local interpretable model agonstic explanations
import shap # SHAP values for local interpretability
import seaborn as sns

<font color="red">TBD: 
- Preprocessing of data
    - Delivery Delay?
    - What features to include/leave out entirely?
    - Missing values in Delay?
    
- Comparison of categorical/binary/continuous variables in LIME

- Normalized graphs -> maybe percentages? 

</font>

In [ ]:
# Separator is ;
data = pd.read_csv("train.csv", sep = ";")

---

## Preprocessing 

In [ ]:
# Draw distribution of target90 variable in data
plt.figure(figsize=(6, 4))

# Count the occurrences of target category
counts = data['target90'].value_counts(normalize=True).reset_index(name='count')

# Plot the count plot
sns.barplot(x='target90', y='count', data=counts, color="#365c8d")

plt.title(f"Distribution of target90")
plt.xlabel('target90')
plt.ylabel('Proportion')
plt.show()

In [ ]:
print(f"Only {np.round(data['target90'].mean()*100,2)}% of customers in the data set repurchased in the next 90 days. This makes the data set imbalanced and we have to proceed with caution.")

In [ ]:
nominal_features = ["salutation", "title", "domain", "newsletter", "model", "paymenttype", "deliverytype", "voucher", "gift", "entry", "points", "shippingcosts"] # target90 left out as well 
cardinal_features = ["numberitems", "weight", "remi", "cancel", "used", "w0", "w1", "w2", "w3", "w4", "w5", "w6", "w7", "w8", "w9", "w10"]
ordinal_features = ["case"]

In [ ]:
x_train = 
y_train = 

---

- Visualization

In [ ]:
# Set the style of seaborn
sns.set(style="whitegrid")

# Create a 4x4 subplot layout
fig, axes = plt.subplots(3, 4, figsize=(16, 12))

# Flatten the axes array to simplify indexing
axes = axes.flatten()

# Loop through each nominal feature and create bar plots
for i, feature in enumerate(nominal_features):
    sns.countplot(x=feature, hue='target90', data=data, palette='viridis', ax=axes[i])
    axes[i].set_title(f'Distribution of {feature} with respect to target90')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Count')
    axes[i].legend(title='target90')

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Assuming 'data' is your DataFrame and 'cardinal_features' is the list of features
# Update 'target90' with your actual target column name

# Set the style of seaborn
sns.set(style="whitegrid")

# Create a 4x4 subplot layout
fig, axes = plt.subplots(4, 4, figsize=(16, 12))

# Flatten the axes array to simplify indexing
axes = axes.flatten()

# Loop through each nominal feature and create countplots for quartiles
for i, feature in enumerate(cardinal_features):

    # Sort data according to feature
    data = data.sort_values(feature)

    # Get data in 25% slides
    data_25 = data.iloc[:int(0.25 * len(data))]
    data_50 = data.iloc[int(0.25 * len(data)):int(0.5 * len(data))]
    data_75 = data.iloc[int(0.5 * len(data)):int(0.75 * len(data))]
    data_100 = data.iloc[int(0.75 * len(data)):]

    # Compute share of target90 = 1 in each quartile
    data_25_y_1 = data_25['target90'].mean()
    data_50_y_1 = data_50['target90'].mean()
    data_75_y_1 = data_75['target90'].mean()
    data_100_y_1 = data_100['target90'].mean()

    # Compute share of target90 = 0 in each quartile
    data_25_y_0 = 1 - data_25_y_1
    data_50_y_0 = 1 - data_50_y_1
    data_75_y_0 = 1 - data_75_y_1
    data_100_y_0 = 1 - data_100_y_1

    # Create a DataFrame with the computed values
    df = pd.DataFrame({'target90': ['0', '1'], 'Q1': [data_25_y_0, data_25_y_1], 'Q2': [data_50_y_0, data_50_y_1],
                        'Q3': [data_75_y_0, data_75_y_1], 'Q4': [data_100_y_0, data_100_y_1]})
    # Plot the results
    sns.barplot(x='target90', y='value', hue='variable', data=pd.melt(df, ['target90']), ax=axes[i], palette = 'viridis')
    axes[i].set_title(f'Distribution of {feature} with respect to target90')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Proportion')
    axes[i].legend(title='Quartile')
    

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Assuming 'data' is your DataFrame and 'cardinal_features' is the list of features
# Update 'target90' with your actual target column name

# Set the style of seaborn
sns.set(style="whitegrid")

# Create a 4x4 subplot layout
fig, axes = plt.subplots(4, 4, figsize=(16, 12))

# Flatten the axes array to simplify indexing
axes = axes.flatten()

# Loop through each nominal feature and create countplots for quartiles
for i, feature in enumerate(cardinal_features):

    # Sort data according to feature
    data = data.sort_values(feature)

    # Get data in 25% slides
    data_25 = data.iloc[:int(0.25 * len(data))]
    data_50 = data.iloc[int(0.25 * len(data)):int(0.5 * len(data))]
    data_75 = data.iloc[int(0.5 * len(data)):int(0.75 * len(data))]
    data_100 = data.iloc[int(0.75 * len(data)):]

    # Compute share of target90 = 1 in each quartile
    data_25_y_1 = data_25['target90'].mean()
    data_50_y_1 = data_50['target90'].mean()
    data_75_y_1 = data_75['target90'].mean()
    data_100_y_1 = data_100['target90'].mean()

    # Compute share of target90 = 0 in each quartile
    data_25_y_0 = 1 - data_25_y_1
    data_50_y_0 = 1 - data_50_y_1
    data_75_y_0 = 1 - data_75_y_1
    data_100_y_0 = 1 - data_100_y_1

    # Create a DataFrame with the computed values
    df = pd.DataFrame({'Quartile': ['Q1', 'Q2', 'Q3', 'Q4'],
                       'target90=0': [data_25_y_0, data_50_y_0, data_75_y_0, data_100_y_0],
                       'target90=1': [data_25_y_1, data_50_y_1, data_75_y_1, data_100_y_1]})

    # Melt the DataFrame for better visualization
    melted_df = pd.melt(df, id_vars='Quartile', var_name='target90', value_name='Proportion')

    # Plot the results
    sns.barplot(x='Quartile', y='Proportion', hue='target90', data=melted_df, ax=axes[i], palette='viridis')
    axes[i].set_title(f'Distribution of {feature} with respect to target90')
    axes[i].set_xlabel('Quartile')
    axes[i].set_ylabel('Proportion')
    axes[i].legend(title='target90')

# Adjust layout for better spacing
plt.tight_layout()
plt.show()


---

## Setup

In [ ]:
# Separator is ;
data = pd.read_csv("train.csv", sep = ";")

In [ ]:
def evaluate_model(y_train, y_train_pred, y_test, y_test_pred):

    # Convert y_train & y_test to np.array, as predictions will also be np.array
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    # Initialize variables for training set
    TP_train, FP_train, TN_train, FN_train = 0, 0, 0, 0

    # Initialize variables for test set
    TP_test, FP_test, TN_test, FN_train = 0, 0, 0, 0

    # Calculate training set scores
    TP_train = np.sum((y_train == 1) & (y_train_pred == 1))
    FP_train = np.sum((y_train == 0) & (y_train_pred == 1))
    TN_train = np.sum((y_train == 0) & (y_train_pred == 0))
    FN_train = np.sum((y_train == 1) & (y_train_pred == 0))

    # Calculate test set scores
    TP_test = np.sum((y_test == 1) & (y_test_pred == 1))
    FP_test = np.sum((y_test == 0) & (y_test_pred == 1))
    TN_test = np.sum((y_test == 0) & (y_test_pred == 0))
    FN_test = np.sum((y_test == 1) & (y_test_pred == 0))
    

    # Calculate training set metrics
    accuracy_train = (TP_train + TN_train) / (TP_train + TN_train + FP_train + FN_train) if (TP_train + TN_train + FP_train + FN_train) != 0 else 0
    precision_train = TP_train / (TP_train + FP_train) if (TP_train + FP_train) != 0 else 0
    sensitivity_train = TP_train / (TP_train + FN_train) if (TP_train + FN_train) != 0 else 0
    specificity_train = TN_train / (TN_train + FP_train) if (TN_train + FP_train) != 0 else 0
    f1_train = 2*TP_train / (2*TP_train + FP_train + FN_train) if (2*TP_train + FP_train + FN_train) != 0 else 0


    # Calculate test set metrics
    accuracy_test = (TP_test + TN_test) / (TP_test + TN_test + FP_test + FN_test) if (TP_test + TN_test + FP_test + FN_test) != 0 else 0
    precision_test = TP_test / (TP_test + FP_test) if (TP_test + FP_test) != 0 else 0
    sensitivity_test = TP_test / (TP_test + FN_test) if (TP_test + FN_test) != 0 else 0
    specificity_test = TN_test / (TN_test + FP_test) if (TN_test + FP_test) != 0 else 0
    f1_test = 2*TP_test / (2*TP_test + FP_test + FN_test) if (2*TP_test + FP_test + FN_test) != 0 else 0

    # Information about distribution of classes in training and test set
    print('Ground truth in training set: \n')
    # Number (and share) of returning customers in training data
    print(f'Number of returning customers training data: {y_train.sum()} ({np.round(y_train.mean()*100,2)}%)')
    # Number (and share) of non-returning customers in training data
    print(f'Number of non-returning customers training data: {len(y_train) - y_train.sum()} ({np.round((1 - y_train.mean())*100,2)}%)')
    # Number (and share) of returning customers in test data
    print(f'Number of returning customers in test data: {y_test.sum()} ({np.round(y_test.mean()*100,2)}%)')
    # Number (and share) of non-returning customers in test data
    print(f'Number of non-returning customers in test data: {len(y_test) - y_test.sum()} ({np.round((1 - y_test.mean())*100,2)}%)')

   # Collect results in a dataframe 
    results_df = pd.DataFrame({
        'Set': ['Training', 'Test'],
        'Accuracy': [accuracy_train, accuracy_test],
        'Precision': [precision_train, precision_test], # Share of positives correctly specified among all predicted positives
        'Sensitivity': [sensitivity_train, sensitivity_test], # Share of actual true positive values found
        'Specificity': [specificity_train, specificity_test], # Share of actual true negative values found
        'TP': [TP_train, TP_test],
        'FP': [FP_train, FP_test],
        'TN': [TN_train, TN_test],
        'FN': [FN_train, FN_test],
        'F1': [f1_train, f1_test]
    })

    return results_df 

In [ ]:
def expected_revenue(y, y_pred):
    """
    Calculate the expected revenue based on target labels in y and model predictions.

    Parameters:
    - y (numpy array or pandas Series): True labels indicating returning buyers (1) or non-returning buyers (0).
    - y_pred (numpy array or pandas Series): Model predictions returning buyers (1) or non-returning buyers (0).

    Returns:
    None

    Prints the expected revenue if all customers were sent a voucher and the expected revenue according to a given model prediction.
    """
    # Just in case
    y = np.array(y)
    y_pred = np.array(y_pred)
    # Compute number of total customers in y
    total_customers = len(y)
    # Compute number of returning buyers in y
    rep_buyers = y.sum()
    # Compute number of non-returning buyers in y
    non_buyers = total_customers - rep_buyers
    # Compute expected revenue if all customers were sent a voucher
    expected_revenue1 = non_buyers * 1.5 - rep_buyers * 5 # Revenue gain - loss
    print(f"Expected revenue if all customers were sent a voucher: €{expected_revenue1}")

    # Compute TN and FN for model predictions
    TN = np.sum((y == 0) & (y_pred == 0))
    FN = np.sum((y == 1) & (y_pred == 0))
    # Compute revenue gain and loss 
    rev_gain = TN * 1.5
    rev_loss = FN * 5
    expected_revenue2 = rev_gain - rev_loss
    percentage_gain = ((expected_revenue2 - expected_revenue1) / expected_revenue1) * 100
    print(f"Expected Revenue according to model: €{expected_revenue}, this is a percentage gain of: %{percentage_gain}")

In [ ]:
# Function to check for NAs in every column
def count_na(df):
    for col in df.columns:          # Loop over all columns
        n_na = df[col].isna().sum() # Count occurrences of missing values
        if n_na > 0:                # Only give column and count if there actually are NAs
            print(col, n_na)        # Print column name and number of NAs

# Apply function
count_na(data)

---

### RandomForestClassifier

- First hyperparameter-tuning run with RandomizedSearchCV

In [ ]:
# Setup ranges for different parameters for hyperparameter tuning
max_depth = range(10,30) 
min_samples_split = range(10,20)
min_samples_leaf = range(5,20)
n_estimators = range(80,150)
criterion = ['gini', 'log_loss', 'entropy']
class_weight = [{1: 5, 0: 1}, 'balanced' ]

# Collect in dictionary
param_dist = {'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf,
               'n_estimators': n_estimators, 'criterion': criterion, 'class_weight': class_weight}

# Set up forest classifier
forest = RandomForestClassifier(bootstrap = True, random_state = seed)

# Set up RandomizedSearchCV
forest_cv = RandomizedSearchCV(forest, param_dist, n_jobs = -1, verbose = 1, n_iter = n_iterations, cv = 5, scoring = "balanced_accuracy", random_state = seed) # suited for imbalanced data sets 

# Fit it to the data
forest_cv.fit(x_train,y_train) # does it automatically use best parameters for prediction afterwards?

# Takes 100 min to run with 1000 iterations and cv = 5

In [ ]:
# Get model predictions for training set
y_train_pred = forest_cv.predict(x_train)
# Get model predictions for test set
y_test_pred = forest_cv.predict(x_test)
# Evaluate performance of cross-validated model
evaluate_model(y_train, y_train_pred, y_test, y_test_pred)

- Second hyperparameter-tuning run around best parameters from previous run

In [ ]:
# Grab optimal parameters from previous CV
best_depth = forest_cv.best_params_["max_depth"]
best_split = forest_cv.best_params_["min_samples_split"]
best_leaf = forest_cv.best_params_["min_samples_leaf"]
best_est = forest_cv.best_params_["n_estimators"]


# Set range around previous optimal parameters to search for even better parameters
max_depth = range(best_depth - 3, best_depth + 3)
min_samples_split = range(best_split - 3, best_split + 3)
min_samples_leaf = range(best_leaf - 3, best_leaf + 3)
n_estimators = range(best_est - 5, best_est + 5)

# Collect in dicionary
param_dist = {'max_depth': max_depth,'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf, 'n_estimators': n_estimators}

# Set up forest
forest = RandomForestClassifier(criterion = forest_cv.best_params_["criterion"], class_weight = 'balanced', bootstrap = True, random_state = seed)
# Set up RandomizedSearchCV
forest_cv2 = RandomizedSearchCV(forest, param_dist, n_jobs = -1, cv = 5,verbose = 1, n_iter = n_iterations, scoring = "balanced_accuracy", random_state = seed)
# Fit it to the data
forest_cv2.fit(x_train, y_train) 

In [ ]:
# Get predictions for training data
y_train_pred_cv2 = forest_cv2.predict(x_train)
# Get predictions for test data
y_test_pred_cv2 = forest_cv2.predict(x_test)
# Evalute performance
evaluate_model(y_train, y_train_pred_cv2, y_test, y_test_pred_cv2)

In [ ]:
# Expected revenue of first RF model 
expected_revenue(y_test, forest_cv.predict(x_test))

In [ ]:
# Expected revenue of second RF model 
expected_revenue(y_test, forest_cv2.predict(x_test)) # better than first RF model 

---

### Ada Boost

In [ ]:
# Set number of parameter sets to try
n_iterations = 50

In [ ]:
# Setup ranges for different parameters for hyperparameter tuning
n_estimators = range(10,50)
learning_rate = np.arange(0.5, 10, 0.25) # In Adaboost learning rate can go to infinity (weights of previous misclassifications)
estimator = [DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=2), DecisionTreeClassifier(max_depth=3)] 
algorithm = ['SAMME', 'SAMME.R']
# No option to set class_weight in AdaBoostClassifier

# Collect in dictionary
param_dist = {'n_estimators': n_estimators, "learning_rate": learning_rate, "estimator": estimator, "algorithm": algorithm}

# Set up AdaBoost classifier
ada = AdaBoostClassifier(random_state = seed)
# Set up RandomizedSearchCV
ada_cv = RandomizedSearchCV(ada, param_dist, n_jobs = -1, verbose = 1, n_iter = n_iterations, cv = 5, scoring = 'balanced_accuracy', random_state = seed)
# Fit it to the data
ada_cv.fit(x_train,y_train) 

In [ ]:
# Get predictions for training data
y_train_pred_ada_cv = ada_cv.predict(x_train)

# Get predictions for test data
y_test_pred_ada_cv = ada_cv.predict(x_test)

# Evaluate performance
evaluate_model(y_train, y_train_pred_ada_cv, y_test, y_test_pred_ada_cv)

- Second hyperparameter tuning run for AdaBoostClassifier

In [ ]:
# Grab optimal parameters from previous CV
best_n_estimators = ada_cv.best_params_["n_estimators"]
best_learningrate = ada_cv.best_params_["learning_rate"]
estimator = ada_cv.best_params_["estimator"]
algorithm = ada_cv.best_params_["algorithm"]

# Set range around previous optimal parameters to search for even better parameters
n_estimators = range(best_n_estimators - 10, best_n_estimators + 10)
learning_rate = np.arange(best_learningrate - 2, best_learningrate + 2, 0.1)

# Collect in dicionary
param_dist = {'n_estimators': n_estimators, "learning_rate": learning_rate}

# Set up forest
ada = AdaBoostClassifier(estimator = estimator, algorithm = algorithm, random_state = seed)
# Set up RandomizedSearchCV
ada_cv2 = RandomizedSearchCV(ada, param_dist, n_jobs = -1, cv = 5, verbose = 1, n_iter = n_iterations, scoring = "balanced_accuracy", random_state = seed)
# Fit it to the data
ada_cv2.fit(x_train, y_train)

In [ ]:
# Get predictions for training data
y_train_pred_ada_cv2 = ada_cv2.predict(x_train)
# Get predictions for test data
y_test_pred_ada_cv2 = ada_cv2.predict(x_test)
# Evalute performance
evaluate_model(y_train, y_train_pred_ada_cv2, y_test, y_test_pred_ada_cv2)

In [ ]:
# Expected revenue of second AdaBoost model 
expected_revenue(y_test, ada_cv.predict(x_test))

In [ ]:
# Expected revenue of second AdaBoost model 
expected_revenue(y_test, ada_cv2.predict(x_test))

---

### XGBoost

In [ ]:
# Setup parameter distributions for hyperparameter tuning
max_depth = range(1, 4)
n_estimators = range(20,80)
learning_rate = np.linspace(0.1, 1, 9)

param_dist = {"max_depth": max_depth, "n_estimators": n_estimators, "learning_rate": learning_rate}

# Set up GradientBoostingClassifier
xgb = XGBClassifier(random_state = seed)

# Set up RandomizedSearchCV
xgb_cv = RandomizedSearchCV(xgb, param_dist, n_jobs = -1, cv = 5, verbose = 1, n_iter = n_iterations, scoring = 'balanced_accuracy', random_state = seed)

# Fit it to the data
xgb_cv.fit(x_train, y_train)

In [ ]:
# Get predictions for training data
y_train_pred_xgb_cv = xgb_cv.predict(x_train)
# Get predictions for test data
y_test_pred_xgb_cv = xgb_cv.predict(x_test)
# Evaluate model performance
evaluate_model(y_train, y_train_pred_xgb_cv, y_test, y_test_pred_xgb_cv)

In [ ]:
# Setup range of parameters around previous optimal parameters
best_n_estimators = xgb_cv.best_params_["n_estimators"]
best_learningrate = xgb_cv.best_params_["learning_rate"]
best_depth = xgb_cv.best_params_["max_depth"]

# Set range around previous optimal parameters to search for even better parameters
n_estimators = range(best_n_estimators - 5, best_n_estimators + 5)
learning_rate = np.arange(best_learningrate - 0.3, best_learningrate + 0.3, 0.05)
max_depth = range(best_depth - 2, best_depth + 2)

# Collect in dicionary
param_dist = {'n_estimators': n_estimators, "learning_rate": learning_rate, "max_depth": max_depth}

# Set up forest
xgb = XGBClassifier(random_state = seed)
# Set up RandomizedSearchCV
xgb_cv2 = RandomizedSearchCV(xgb, param_dist, n_jobs = -1, cv = 5,verbose = 1, n_iter = n_iterations, scoring = "balanced_accuracy", random_state = seed)
# Fit it to the data
xgb_cv2.fit(x_train, y_train) # does it automatically use best parameters? 

In [ ]:
# Get predictions for training data
y_train_pred_xgb_cv2 = xgb_cv2.predict(x_train)
# Get predictions for test data
y_test_pred_xgb_cv2 = xgb_cv2.predict(x_test)
# Evaluate model performance
evaluate_model(y_train, y_train_pred_xgb_cv2, y_test, y_test_pred_xgb_cv2)

In [ ]:
# Expected revenue of first XGB model
expected_revenue(y_test, xgb_cv.predict(x_test))

In [ ]:
# Expected revenue of second XGB model
expected_revenue(y_test, xgb_cv2.predict(x_test))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import uniform


gb = GradientBoostingClassifier()

# Define parameter grid for RandomizedSearchCV
param_dist_gb = {
    'n_estimators': range(50, 200),
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': range(3, 10),
    'min_samples_split': range(2, 20),
    'min_samples_leaf': range(1, 20),
    'subsample': uniform(0.5, 1.0)
}

# Create RandomizedSearchCV instance
# verbose = n: how many process messages are printed to console
randSearch_gb = RandomizedSearchCV(gb, param_distributions=param_dist_gb, n_iter=50, cv=5,
                                   verbose=0, random_state=seed)

# Fit the RandomizedSearchCV
randSearch_gb.fit(x_train, y_train)

---

# GradientBoostClassifier

---

### Model interpretations

Explain model predictions of best performing model: Random Forest

- Feature importance

In [ ]:
# Set model to best estimator from CV
model = forest_cv2.best_estimator_

# Plot feature importance
fig = plt.figure(figsize=(12, 6))
ax = fig.gca() #get current axis
ax.bar(range(x_train.shape[1]), model.feature_importances_, color = "#0081ff", width = 0.8)
ax.set_xticks(np.arange(x_train.shape[1]))
ax.set_xticklabels([f'{col}' for col in x_train.columns], rotation=45, ha='right')
ax.set_xlabel('Feature', fontsize = 12)
ax.set_ylabel('Feature Importance', fontsize = 12)
ax.set_title('Overall feature importances of Random Forest Classifier')
ax.grid(axis='y', linestyle='--', alpha=0.7)
ax.legend(['Feature Importance'])
plt.tight_layout()
# plt.savefig('feature_importance.png', dpi=300)


- Permutation feature importance

In [ ]:
# Set list of metrics to compute feature importance for
metrics = ['accuracy', 'precision', 'recall', 'f1'] # specificity not a valid scoring metric
# Initialize empty dictionary to store results
pfi_scores = {}
# Loop over all metrics
for metric in metrics: 
    print('Computing permutation importance with {0}...'.format(metric))
    pfi_scores[metric] = permutation_importance(model, x_test, y_test, scoring = metric, n_repeats = 30, random_state = seed)

In [ ]:
# Visualization of results 
features = x_test.columns.values
fig, ax = plt.subplots(nrows = 1, ncols = 5, figsize = (18, 4))

scores = model.feature_importances_
features = x_test.columns.to_numpy()
srtd = np.argsort(-scores) # sort descending and get indices

top = 10 # Top 10 features
ax[0].barh(y = np.arange(0, top), width=scores[srtd[:top]], color='#0081ff', alpha = 0.6) # regular feature importance of RF 
for i in range(top):
    ax[0].text(0.01, i-0.15, features[srtd[i]]) # add feature names to plot
ax[0].set_yticks([])
ax[0].set_ylabel('Top {0} features'.format(top));
ax[0].set_xlabel('Feature Importances')
ax[0].set_title('Computed by Random Forest')

for k, metric in enumerate(metrics):
    scores = pfi_scores[metric]['importances_mean']
    srtd = np.argsort(-scores)
    
    ax[k+1].barh(y=np.arange(0, top), width = scores[srtd[:top]], color = '#0081ff', alpha = 0.6)
    for i in range(top):
        ax[k+1].text(0.001, i-0.15, features[srtd[i]])
    ax[k+1].set_yticks([])
    ax[k+1].set_ylabel('Top {0} features scored with {1}'.format(top, metric));
    ax[k+1].set_xlabel('Permutation Feature Importances')
    ax[k+1].set_title('Scored using {0}'.format(metric))

plt.tight_layout()

##### Global model agnostic methods

- Partial dependence plots

In [ ]:
continuous_features = ["numberitems", "weight", "remi", "cancel", "used", "w0", "w1", "w2", "w3", "w4", "w5", "w6", "w7", "w8", "w9", "w10", ]
ordinal_features = ["case"]
binary_features = ["title", "newsletter", "deliverytype", "voucher", "domain_aol.com", "domain_arcor.de", "domain_freenet.de", "domain_gmail.com", "domain_gmx.de",
                    "domain_hotmail.de", "domain_online.de", "domain_onlinehome.de", "domain_t-online.de", "domain_web.de", "domain_yahoo.com", "domain_yahoo.de", "domain_others",
                    "salutation_Company", "salutation_Mr.", "salutation_Ms.", "model_1", "model_2", "model_3", "paymenttype_Cash", "paymenttype_Credit",
                      "paymenttype_Invoice", "paymenttype_Transfer"]

In [ ]:
# Continuous features
fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(20, 10))

# Iterate through both features and axes
for feature, axis in zip(continuous_features, ax.flatten()):
    pdp.from_estimator(forest_cv2.best_estimator_, x_train,
                       features=[feature],  # Use only one feature at a time
                       feature_names=list(x_train.columns),
                       kind='average', 
                       response_method = "predict_proba", # Use predict_proba to get probability of class 1
                       ax=axis)

    # Set the title for each subplot
    # axis.set_title(feature, fontsize = 12)

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=11, ncols=4, figsize=(20, 10))
# Iterate through both continuous and binary features and axes
for feature, axis in zip(continuous_features + binary_features, ax.flatten()):
    # Check if the feature is continuous or binary
    if feature in continuous_features:
        pdp.from_estimator(forest_cv2, x_train,
                           features=[feature],
                           feature_names=list(x_train.columns),
                           kind='average', 
                           response_method="predict_proba",
                           ax=axis)
    else:  # Binary features
        pdp.from_estimator(forest_cv2, x_train,
                           features=[feature],
                           feature_names=list(x_train.columns),
                           kind='both',  
                           response_method="predict_proba",
                           ax=axis)

    # Set the title for each subplot
    #axis.set_title(feature, fontsize=12)

- Global surrogate model

In [ ]:
# Compute the model
yb_train_pred = forest_cv2.predict(x_train)  # Training set predictions of the black-box (RF) model
yb_test_pred = forest_cv2.predict(x_test)  # Test set predictions of the black-box (RF) model

acc={}

# Loop over max_depth
depth_limits = [2, 3, 4, 5, 6, 7, 8, 9, 10]
for max_depth in depth_limits:
    print('Surrogate max-depth = {0}'.format(max_depth))
    surrogate = DecisionTreeClassifier(max_depth = max_depth, criterion = 'gini', 
                                       min_samples_leaf = 20, class_weight = 'balanced', random_state = seed)
    surrogate.fit(x_train, yb_train_pred)
    ys_train_pred = surrogate.predict(x_train)
    ys_test_pred = surrogate.predict(x_test)
       
    acc[max_depth] = {'trn': {'black-box': accuracy_score(y_train, yb_train_pred),
                                   'surrogate': accuracy_score(y_train, ys_train_pred),
                                   'r2': r2_score(yb_train_pred, ys_train_pred)},
                           'tst': {'black-box': accuracy_score(y_test, yb_test_pred),
                                   'surrogate': accuracy_score(y_test, ys_test_pred),
                                   'r2': r2_score(yb_test_pred, ys_test_pred)}} # r_squared as in regression 

In [ ]:
# Visualization of results
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

fig_labels = ['Training Set', 'Testing Set']
markers = [None, 's', 'o']
for i, dset in enumerate(['trn', 'tst']):
    for j, curve in enumerate(['black-box', 'surrogate', 'r2']):
        z = [acc[mleaf][dset][curve] for mleaf in depth_limits]
        ax[i].plot(depth_limits, z, marker=markers[j])
    ax[i].legend(['Black-box accuracy', 'Surrogate accuracy', 'R2 Black-box vs. Surrogate'])
    ax[i].set_title(fig_labels[i])
    ax[i].set_xlabel('Maximum depth of decision tree')
    ax[i].set_ylabel('accuracy / r2 score')
    ax[i].set_xticks(depth_limits)
plt.tight_layout()

In [ ]:
# Set up surrogate model as decision tree with max_depth = 6
surrogate = DecisionTreeClassifier(max_depth = 6, criterion = 'gini', min_samples_leaf = 20, class_weight = 'balanced', random_state = seed)
surrogate.fit(x_train, yb_train_pred)

from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(15,6), dpi=300)
ax = fig.gca()
plot_tree(surrogate, fontsize = 3, feature_names = x_train.columns.values, 
          rounded=True, filled=True, ax = ax, class_names = ["No Repurchase", "Repurchase"]) # in order of surrogate.classes_
fig.savefig('Regression Tree.PNG')

##### Local model agnostic methods (LIME)

- Explain one prediction instance of our trained model

In [ ]:
cat_features = ['title', 'newsletter', 'deliverytype', 'invoicepostcode', 'voucher',
                'case', 'gift', 'entry', 'points', 'shippingcosts', 'paymenttype',
                'salutation', 'delay']

cat_idx = np.array([cat_features.index(f) for f in cat_features])


lime_explainer = lime_tabular.LimeTabularExplainer(x_train.values,
                                              feature_names=list(x_train.columns), 
                                              class_names=['No purchase', 'Repurchase'], 
                                              categorical_features=cat_idx,
                                              kernel_width=75.0,
                                              categorical_names=cat_features,
                                              discretize_continuous=False)

In [ ]:
# Explain a single instance using example index:
exp = lime_explainer.explain_instance(x_test.iloc[3000], forest_cv2.predict_proba)
exp.show_in_notebook(show_table=False, show_all=False)